In [1]:
import pandas as pd
import os
from unidecode import unidecode
import warnings
warnings.filterwarnings('ignore')

# Definir rutas de los archivos:

In [2]:
project_directory = os.path.abspath('../')
tiempos_directory = os.path.join(project_directory, 'data', 'tiempos_logisticos')
tiempos_files = os.listdir(tiempos_directory)
tiempos_files

['RemesasRNDC_202207.txt',
 'RemesasRNDC_202208.txt',
 'RemesasRNDC_202209.txt',
 'RemesasRNDC_202210.txt',
 'RemesasRNDC_202211.txt',
 'RemesasRNDC_202212.txt',
 'RemesasRNDC_202301.txt',
 'RemesasRNDC_202302.txt',
 'RemesasRNDC_202303.txt']

# Exploración archivos:

## Reducción de columnas:

Para optimizar el uso del espacio en el bucket de data refinada en S3 y procesar solo la data necesaria en Lambda, se procede a identificar las columnas que no serán utilizadas.

### Definir campos comunes entre los archivos:

La lista resultante de campos será la primera base para definir los campos a guardar en el bucket de data refinada. En este caso se cambian algunos de los nombres de los campos para que coincidan con el nombre de los campos en los archivos de estadísticas.

In [18]:
ls_raw_fields = []
file_encoding = 'latin-1'

for file in tiempos_files:
    file_path = os.path.join(tiempos_directory, file)
    df = pd.read_csv(file_path,nrows=1,delimiter='|',encoding=file_encoding)
    ls_raw_fields_aux = df.columns.to_list()
    if len(ls_raw_fields)==0:
        ls_raw_fields = ls_raw_fields_aux
    else:
        list(set(ls_raw_fields).intersection(ls_raw_fields_aux))
ls_raw_fields

['AÑOMES',
 'NATURALEZA',
 'COD_PRODUCTO',
 'PRODUCTO',
 'CANTIDAD',
 'UNID_MEDIDA',
 'FECHASALIDACARGUE',
 'HORA_SALIDA_CARGUE',
 'FECHALLEGADADESCARGUE',
 'HORA_LLEGADA_DESCARGUE',
 'CODIGO_CARGUE',
 'CARGUE',
 'CODIGO_DESCARGUE',
 'DESCARGUE',
 'HORAS_VIAJE',
 'HORAS_ESPERA_CARGUE',
 'HORAS_CARGUE',
 'HORAS_ESPERA_DESCARGUE',
 'HORAS_DESCARGUE',
 'VALOR_PACTADO',
 'VALOR_PAGADO',
 'CANTIDAD_REMESAS_VIAJE',
 'EMPRESA_TRANSPORTE',
 'PLACA',
 'CONFIGURACION',
 'CONDUCTOR']

### Definir campos finales:

Se considera a los campos que no están en esta lista como redundates porque no harán parte del esquema final del modelo. En este caso solo son necesarios los campos que hacen referencia a los tiempos losgísticos, origen, destino, tipo de vehículo y naturaleza de la carga. Estos serán los campos a consultar, explorar y guardar en el bucket de data refinada.

In [19]:
ls_refined_fields = ['AÑOMES','NATURALEZA','CONFIGURACION','CODIGO_CARGUE','CODIGO_DESCARGUE','HORAS_VIAJE','HORAS_ESPERA_CARGUE',
                     'HORAS_CARGUE','HORAS_ESPERA_DESCARGUE','HORAS_DESCARGUE']

## Identificar y definir tipos de datos:

El objetivo es identificar el tipo de datos presente en todos los archivos y saber si corresponden entre archivo y archivo. Estos serán el tipo de campos a asegurar durante el proceso de limpieza.

In [21]:
ls_raw_fields = []
df_info = pd.DataFrame()
for file in tiempos_files:
    file_path = os.path.join(tiempos_directory, file)
    df = pd.read_csv(file_path,delimiter='|',encoding=file_encoding,usecols=ls_refined_fields)
    ls_columns = df.columns.tolist()
    ls_data_types = df.dtypes.tolist()
    df_info_aux = pd.DataFrame({'column_name': ls_columns, 'data_type': ls_data_types})
    if df_info.empty:
        df_info = df_info_aux
    else:
        df_info = pd.merge(df_info,df_info_aux,on = ['column_name','data_type'],how='left')

val =  len(df_info) == len(ls_refined_fields)
print(df_info)
print(val)

              column_name data_type
0                  AÑOMES     int64
1              NATURALEZA    object
2           CODIGO_CARGUE     int64
3        CODIGO_DESCARGUE     int64
4             HORAS_VIAJE   float64
5     HORAS_ESPERA_CARGUE   float64
6            HORAS_CARGUE   float64
7  HORAS_ESPERA_DESCARGUE   float64
8         HORAS_DESCARGUE   float64
9           CONFIGURACION    object
True


### Se crea un diccionario con el tipo de datos:

Este diccionario se usará en el proceso de lectra de los archivos para asegurar el formato de los campos.

In [23]:
dict_types = df_info.set_index('column_name')['data_type'].to_dict()
# df = pd.read_excel(file_path,nrows=100,usecols=ls_refined_fields,dtype=dict_types)
dict_types

{'AÑOMES': dtype('int64'),
 'NATURALEZA': dtype('O'),
 'CODIGO_CARGUE': dtype('int64'),
 'CODIGO_DESCARGUE': dtype('int64'),
 'HORAS_VIAJE': dtype('float64'),
 'HORAS_ESPERA_CARGUE': dtype('float64'),
 'HORAS_CARGUE': dtype('float64'),
 'HORAS_ESPERA_DESCARGUE': dtype('float64'),
 'HORAS_DESCARGUE': dtype('float64'),
 'CONFIGURACION': dtype('O')}

## Validaciones:

En este paso se establecen las validaciones de los archivos, en caso de que algún archivo no supere la fase de validación no será cargado en el bucket de data refinada.

### Validar formato de archivos:

Se valida si el formato de los archivos

In [24]:
for file in tiempos_files:
    if not file_path.endswith('.txt'):
        val = False
    else:
        val = True
    print(f'file: {file} | validation: {val}')

file: RemesasRNDC_202207.txt | validation: True
file: RemesasRNDC_202208.txt | validation: True
file: RemesasRNDC_202209.txt | validation: True
file: RemesasRNDC_202210.txt | validation: True
file: RemesasRNDC_202211.txt | validation: True
file: RemesasRNDC_202212.txt | validation: True
file: RemesasRNDC_202301.txt | validation: True
file: RemesasRNDC_202302.txt | validation: True
file: RemesasRNDC_202303.txt | validation: True


###  Validar campos:

Se valida si todos los campos definidos están presentes en el archivo.

In [25]:
for file in tiempos_files:
    file_path = os.path.join(tiempos_directory, file)
    df = pd.read_csv(file_path,nrows=0,delimiter='|',encoding=file_encoding)
    val = all(elem in df.columns.to_list() for elem in ls_refined_fields)
    print(f'file {file} | validation: {val}')

file RemesasRNDC_202207.txt | validation: True
file RemesasRNDC_202208.txt | validation: True
file RemesasRNDC_202209.txt | validation: True
file RemesasRNDC_202210.txt | validation: True
file RemesasRNDC_202211.txt | validation: True
file RemesasRNDC_202212.txt | validation: True
file RemesasRNDC_202301.txt | validation: True
file RemesasRNDC_202302.txt | validation: True
file RemesasRNDC_202303.txt | validation: True


### Validar coherencia en los datos:

Se valida si el periodo al que hace referencia el nombre del archivo corresponde con el dato de la columna "MES". Para el momento de la limpieza si el archivo no cumple esta validación no será procesado ni cargado en el bucked de data refinada.

In [27]:
for file in tiempos_files:
    file_path = os.path.join(tiempos_directory, file)
    period = float(file.split('_')[-1].split('.')[0])
    df = pd.read_csv(file_path,usecols=['AÑOMES'],delimiter='|',encoding=file_encoding)
    period_avg = df.AÑOMES.mean()
    val = period == period_avg
    print(f'file: {file} | period: {period} | period_avg: {period_avg} | validation: {val}')

file: RemesasRNDC_202207.txt | period: 202207.0 | period_avg: 202207.0 | validation: True
file: RemesasRNDC_202208.txt | period: 202208.0 | period_avg: 202208.0 | validation: True
file: RemesasRNDC_202209.txt | period: 202209.0 | period_avg: 202209.0 | validation: True
file: RemesasRNDC_202210.txt | period: 202210.0 | period_avg: 202210.0 | validation: True
file: RemesasRNDC_202211.txt | period: 202211.0 | period_avg: 202211.0 | validation: True
file: RemesasRNDC_202212.txt | period: 202212.0 | period_avg: 202212.0 | validation: True
file: RemesasRNDC_202301.txt | period: 202301.0 | period_avg: 202301.0 | validation: True
file: RemesasRNDC_202302.txt | period: 202302.0 | period_avg: 202302.0 | validation: True
file: RemesasRNDC_202303.txt | period: 202303.0 | period_avg: 202303.0 | validation: True


# Transformaciones y limpieza:

En este paso entrarán todos los archivos que hayan pasado la fase de validación. Sin embargo, para el ejemplo de este notebook ssolo se usará el archivo "RemesasRNDC_202207.txt". En este caso se cambiará el nombre de varias columnas para que coincidan con los campos de los archivos de estadísticas.

In [39]:
file = 'RemesasRNDC_202207.txt'
file_path = os.path.join(tiempos_directory, file)
df = pd.read_csv(file_path,usecols=ls_refined_fields,delimiter='|',encoding=file_encoding,dtype=dict_types)
df.rename(columns={'AÑOMES': 'ANOMES','NATURALEZA':'NATURALEZACARGA','CONFIGURACION':'COD_CONFIG_VEHICULO',
                    'CODIGO_CARGUE':'CODMUNICIPIOORIGEN','CODIGO_DESCARGUE':'CODMUNICIPIODESTINO'},inplace=True)
df.drop_duplicates(inplace=True) # eliminar duplicados
df = df.applymap(lambda x: x.upper() if isinstance(x, str) else x) # volver mayúsculas los campos str
df = df.applymap(lambda x: unidecode(x) if isinstance(x, str) else x) # quitar acentos a campos str
df_grouped = df.groupby(by=['ANOMES','NATURALEZACARGA','CODMUNICIPIOORIGEN','CODMUNICIPIODESTINO','COD_CONFIG_VEHICULO'],as_index=False).mean()
print(f'file: {file} | status: refined')

file: RemesasRNDC_202207.txt | status: refined


In [40]:
df_grouped

,ANOMES,NATURALEZACARGA,CODMUNICIPIOORIGEN,CODMUNICIPIODESTINO,COD_CONFIG_VEHICULO,HORAS_VIAJE,HORAS_ESPERA_CARGUE,HORAS_CARGUE,HORAS_ESPERA_DESCARGUE,HORAS_DESCARGUE
0,202207,CARGA EXTRADIMENSIONADA,5154000,23555000,3S2,97.92,0.08,1.98,0.07,2.00
1,202207,CARGA EXTRADIMENSIONADA,5234000,11001000,3S3,122.50,0.00,0.98,0.00,1.00
2,202207,CARGA EXTRADIMENSIONADA,11001000,73770000,3S3,28.40,0.13,1.58,0.17,1.42
3,202207,CARGA EXTRADIMENSIONADA,20011000,73547000,3S3,99.92,0.17,2.40,0.08,2.15
4,202207,CARGA EXTRADIMENSIONADA,20770000,68615031,3S3,33.83,0.17,1.98,0.17,1.98
...,...,...,...,...,...,...,...,...,...,...
59471,202207,SEMOVIENTES,73411000,73443000,2,9.00,0.00,0.98,0.00,0.98
59472,202207,SEMOVIENTES,73443000,73283000,2,9.00,0.00,0.98,0.00,0.98
59473,202207,SEMOVIENTES,73449000,73352000,2,14.50,0.00,0.98,0.00,0.98
59474,202207,SEMOVIENTES,73449000,73873000,2,12.00,0.00,0.98,0.00,0.98
